In [1]:
import os
import gc
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

import data_sampling_workflow.sample_rasters as sr
from rasterio.crs import CRS

import planetary_computer as pc

from skimage.feature import graycomatrix, graycoprops
import raster_to_features as rf

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

window_r = 1  #in pixels

distances = [1]
angles = [np.pi/2]
props = ['contrast']#, 'correlation']

# ***************************************************

In [3]:
itemids = pd.read_csv(sr.path_to_aoi_itemids_csv())
itemids = itemids[ (itemids.aoi_name == 'point_conception') & (itemids.year == 2020)]
itemid = itemids.itemid[0]
raster = rf.rioxr_from_itemid(itemid)

In [4]:
# BENCHAMRK: ITERATING THROUGH EACH PIXEL
y_len = 100
x_len = 100
band_n = 0


t0 = time.time()
contrast2 = np.zeros((y_len,x_len))
for y in range(y_len):
    for x in range(x_len):
        window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]      # MAKE THIS A DIRECT COMPARISON BEFORE MAKING THE WINDOW OR START x,y AT RIGHT INDICES
        if (window.shape[1] != 2*window_r+1) | (window.shape[2] != 2*window_r+1):
            contrast2[y,x] = np.nan
        else:
            glcm = graycomatrix(window[band_n], distances=distances, angles=angles)
#            print(np.nonzero(glcm), '\n')
            contrast2[y,x] = graycoprops(glcm, 'contrast')[0,0]
            
print(time.time() - t0)

25.16366720199585


In [5]:
#The value P[i,j,d,theta] is the number of times that gray-level j occurs at a distance d and at an angle theta from gray-level i.

def L_out_R_in(raster, x,y, prev_glcm, window_r):
#    x,y = column and row in raster of center of window over which prev_glcm was calculated NOT
    new = prev_glcm   # this is 4D array of shape 256, 256, 1, 1 (assuming 1 direction-N and 1 angle pi/2)
    top = y - window_r
    bott = y + window_r
        
    # get i,j occurrences in Lcolumn... 
    #Lx = x - window_r
    Lx = x - window_r -1
#    pairs = np.column_stack((raster[top+1:bott+1 ,Lx], raster[top:bott ,Lx]))
    pairs = np.column_stack((raster[top:bott ,Lx],raster[top+1:bott+1 ,Lx]))
    # print('out')
    # print(pairs, '\n')
    u_pairs, counts = np.unique(pairs, return_counts=True, axis=0)
    # ... and substract them 
    for pair, count in zip(u_pairs, counts):
        new[pair[0],pair[1],0,0] -= count
        
    # get i,j occurrences in Rcolumn...     
#    Rx = x + window_r+1
    Rx = x + window_r
    pairs = np.column_stack((raster[top+1:bott+1 ,Rx], raster[top:bott ,Rx]))
    # print('in')
    # print(pairs, '\n')    
    u_pairs, counts = np.unique(pairs, return_counts=True, axis=0)
    # ... and add them 
    for pair, count in zip(u_pairs, counts):
        new[pair[0],pair[1],0,0] += count

    return new
    

In [6]:
y_len = 100
x_len = 100
contrast = np.zeros((y_len,x_len))

t0 = time.time()

for y in range(y_len):    
    for x in range(x_len):
        window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]      # MAKE THIS A DIRECT COMPARISON BEFORE MAKING THE WINDOW OR START x,y AT RIGHT INDICES
        
        if (window.shape[1] != 2*window_r+1) | (window.shape[2] != 2*window_r+1):
            contrast[y,x] = np.nan
        else:
 #           print(window[band_n].data, '\n')
            if x == window_r:
                this_glcm = graycomatrix(window[band_n].squeeze(), distances=distances, angles=angles)
            else:
                this_glcm = L_out_R_in(raster[band_n].squeeze(), x, y, prev_glcm, window_r)
#            print(np.nonzero(this_glcm), '\n')                
            contrast[y,x] = graycoprops(this_glcm, 'contrast')[0,0]
            prev_glcm = this_glcm
print(time.time() - t0)

57.16921615600586


In [7]:
contrast2

array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan, 121.83333333,  34.        , ..., 165.66666667,
        126.16666667, 136.5       ],
       [         nan, 213.83333333, 126.        , ..., 221.66666667,
        198.16666667, 180.33333333],
       ...,
       [         nan,  24.        ,  41.33333333, ..., 105.83333333,
         55.33333333,  83.33333333],
       [         nan,  74.        ,  42.5       , ..., 136.33333333,
        280.66666667, 378.16666667],
       [         nan,  58.5       ,  21.        , ..., 103.66666667,
        241.83333333, 361.33333333]])

In [8]:
contrast

array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan, 121.83333333,  34.        , ..., 234.68944099,
        234.7607362 , 233.68902439],
       [         nan, 213.83333333, 126.        , ..., 280.32919255,
        279.8404908 , 278.43636364],
       ...,
       [         nan,  24.        ,  41.33333333, ..., 240.2375    ,
        240.98757764, 238.27607362],
       [         nan,  74.        ,  42.5       , ..., 392.37572254,
        390.94827586, 386.55113636],
       [         nan,  58.5       ,  21.        , ..., 467.94767442,
        463.41954023, 458.19886364]])

In [9]:
contrast-contrast2

array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,   0.        ,   0.        , ...,  69.02277433,
        108.59406953,  97.18902439],
       [         nan,   0.        ,   0.        , ...,  58.66252588,
         81.67382413,  98.1030303 ],
       ...,
       [         nan,   0.        ,   0.        , ..., 134.40416667,
        185.6542443 , 154.94274028],
       [         nan,   0.        ,   0.        , ..., 256.04238921,
        110.28160919,   8.3844697 ],
       [         nan,   0.        ,   0.        , ..., 364.28100775,
        221.58620689,  96.8655303 ]])